# A Moving Body Without Forces
**Run all cells and press the play button.**

In [ ]:
# PLEASE BE PATIENT WITH THE FIRST BLOCK
# the kernel needs to load and netgen needs to be imported, which takes its time
from netgen.occ import Box, Pnt

In [ ]:
import piplite
await piplite.install("https://triadtitans.github.io/rigid_body_interactive/static/pyodide/lib_rigid_body-0.0.1-cp312-cp312-pyodide_2024_0_wasm32.whl")
await piplite.install("https://files.pythonhosted.org/packages/93/c1/68423f43bc95d873d745bef8030ecf47cd67f932f20b3f7080a02cff43ca/widgetsnbextension-4.0.11-py3-none-any.whl")
await piplite.install("pythreejs")

from lib_rigid_body.rigid_body_FEM import *
import lib_rigid_body.rigid_body_FEM.bla as bla

In [ ]:
import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

In [2]:
# set up OCC CAD model
# center of mass is automatically moved to origin by body_from_solid
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))

In [3]:
# from ngsolve.webgui import Draw
# Draw(mesh)

In [4]:
def body_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"

    # important: move the center of mass into the origin
    obj = obj.Move((-obj.center[0], -obj.center[1], -obj.center[2]))
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    body = RigidBody_FEM()
    for i in range(3) : body.center[i] = obj.center[i]
    body.mass = obj.mass
    body.inertia = inertia_matrix
    body.recalcMassMatrix()
    
    return body

In [5]:
def extract_vertices(obj: TopoDS_Shape):
    "extracts a p3js compatible vertex list from a netgen.occ TopoDS_Shape"
    
    data = obj._webgui_data()["Bezier_trig_points"]
    
    # for every face, each of the verts arrays holds one vertex
    verts1 = data[0]
    verts2 = data[1]
    verts3 = data[2]
    
    # corresponding normals
    normals1 = data[3]
    normals2 = data[4]
    normals3 = data[5]

    combined_vertices = []
    for i in range(0, len(verts1), 4):
        combined_vertices.append(verts1[i : i+3])
        combined_vertices.append(verts2[i : i+3])
        combined_vertices.append(verts3[i : i+3])

    combined_normals = []
    for i in range(0, len(normals1), 3):
        combined_normals.append(normals1[i : i+3])
        combined_normals.append(normals2[i : i+3])
        combined_normals.append(normals3[i : i+3])
            
    return combined_vertices, combined_normals

In [6]:
view_width = 1000
view_height = 700

# set up pythreejs 3d object
vertices, normals = extract_vertices(box)
buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(vertices), "normal" : p3.BufferAttribute(normals)})
# buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
# n = p3.VertexNormalsHelper(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")

# set up scene
scene = p3.Scene(children=[p3mesh, camera, key_light, ambient_light, grid])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [7]:
# set up physics simulation object for cube
r =  body_from_solid(box)
r.momentumTrans = (0, 0, 0.1)
r.momentumRot = (0, 0.01, 0)

# set up physics simulation environment
rbs = RBS_FEM()
rbs.gravity = (0, 0, 0)
connector = rbs.addBody(r)
rbs.saveState()

In [8]:
print(r.momentumTrans)
print(r.momentumRot)

(0.0, 0.0, 0.1)
(0.0, 0.01, 0.0)


In [9]:
# play/reset button
play = widgets.Play(
    value=0,
    min=0,
    max=10000,
    step=1,
    interval=100/60,
    description="Press play",
    disabled=False
)

In [10]:
p3mesh.matrixAutoUpdate = False # make mesh movable

def update():
    "update function, gets called every timestep; quasi main loop"
    simulate(rbs, 2.5, 2)
    p3mesh.matrix=rbs.bodies()[0].transformation.asTuple()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            rbs.reset()
            p3mesh.matrix=rbs.bodies()[0].transformation.asTuple()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        rbs.reset()
        p3mesh.matrix=rbs.bodies()[0].transformation.asTuple()

play.observe(observer)


In [11]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.4285714285714286, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0…